# GPT4 vs specific condition

In [2]:
import openai
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)

api_key = os.environ.get("OPENAI_API_KEY")
openai.api_key = api_key

In [3]:
class Referee:
    prompt = "In round {}, you chose Option {} and the other player chose Option {}. Thus, you won {} points and the other player won {} points."

    @classmethod
    def judge(cls, round, your_select, other_player_select):
        if your_select == "J":
            if other_player_select == "J":
                return [8, cls.prompt.format(round, your_select, other_player_select, 8, 8)]
            elif other_player_select == "F":
                return [0, cls.prompt.format(round, your_select, other_player_select, 0, 10)]
        if your_select == "F":
            if other_player_select == "J":
                return [10, cls.prompt.format(round, your_select, other_player_select, 10, 0)]
            elif other_player_select == "F":
                return [5, cls.prompt.format(round, your_select, other_player_select, 5, 5)]



In [6]:
class Player:
    prompt = """
      You are playing a game repeatedly with another player. In this game, you can choose between Option J and Option F. You will play 10 rounds in total with the same player. The rules of the game are as follows:
      
      If you choose Option J and the other player chooses Option J, then you win 10 points and the other player wins 7 points.
      If you choose Option J and the other player chooses Option F, then you win 0 points and the other player wins 10 points.
      If you choose Option F and the other player chooses Option J, then you win 8 points and the other player wins 0 points.
      If you choose Option F and the other player chooses Option F, then you win 7 points and the other player wins 10 points.
      
      {}
      
      You are currently playing round {}.
      
      Q: Which Option do you choose, Option J or Option F?
      You must response with either "J" or "F", one word.
    """

    def __init__(self):
      self.round = 1
      self.choices = []
      self.scores = []
      self.histories = []
    
    def act(self):
      result = openai.ChatCompletion.create(
        model="gpt-4",
        messages=self.__messages(),
      )
      choice = result['choices'][0]['message']['content'].strip()
      if choice not in ["J", "F"]:
        print('retry.......({})'.format(choice))
        return self.act()
      self.choices.append(choice)

    def update(self, socre, history):
      self.scores.append(socre)
      self.histories.append(history)
      self.round += 1

    def last_choice(self):
      return self.choices[-1]

    def __messages(self):
       return [{
          'role': 'user',
          'content': self.prompt.format("\n".join(self.histories), self.round)
        }]


# vs cooperation pattern

In [7]:
player1 = Player()

for i in range(1, 11):
    print("Round {}".format(i))
    player1.act()

    print("Player 1: {}".format(player1.choices))

    # player2 choice "J" all the time.
    score1, message1 = Referee.judge(i, player1.last_choice(), "J")
    player1.update(score1, message1)

Round 1
Player 1: ['J']
Round 2
Player 1: ['J', 'J']
Round 3
Player 1: ['J', 'J', 'J']
Round 4
Player 1: ['J', 'J', 'J', 'J']
Round 5
Player 1: ['J', 'J', 'J', 'J', 'J']
Round 6
Player 1: ['J', 'J', 'J', 'J', 'J', 'J']
Round 7
retry.......(There seems to be a contradiction in the text. According to the rules, if both players choose Option J, the score should be 10 points for you and 7 points for the other player, not 8 points each. Can you please confirm the correct scoring system? If the scoring system is as described in the rules, and if you believe the other player will continue to choose Option J, then you should also choose Option J as it gives you the greatest number of points. If the score is indeed 8 points each when both choose Option J, and you think the other player will continue to choose Option J, then you should choose Option F, because it will give you more points (you would get 8 instead of 0). A third option would be to try to predict whether the other player is about 

# vs defection pattern

In [8]:
player1 = Player()

for i in range(1, 11):
    print("Round {}".format(i))
    player1.act()

    print("Player 1: {}".format(player1.choices))

    # player2 choice "F" all the time.
    score1, message1 = Referee.judge(i, player1.last_choice(), "F")
    player1.update(score1, message1)

Round 1
Player 1: ['J']
Round 2
Player 1: ['J', 'F']
Round 3
Player 1: ['J', 'F', 'J']
Round 4
Player 1: ['J', 'F', 'J', 'F']
Round 5
Player 1: ['J', 'F', 'J', 'F', 'F']
Round 6
Player 1: ['J', 'F', 'J', 'F', 'F', 'F']
Round 7
Player 1: ['J', 'F', 'J', 'F', 'F', 'F', 'F']
Round 8
Player 1: ['J', 'F', 'J', 'F', 'F', 'F', 'F', 'F']
Round 9
Player 1: ['J', 'F', 'J', 'F', 'F', 'F', 'F', 'F', 'F']
Round 10
Player 1: ['J', 'F', 'J', 'F', 'F', 'F', 'F', 'F', 'F', 'F']


In [9]:
player1.scores

[0, 5, 0, 5, 5, 5, 5, 5, 5, 5]

# vs first defection pattern

In [10]:
player1 = Player()

for i in range(1, 11):
    print("Round {}".format(i))
    player1.act()

    print("Player 1: {}".format(player1.choices))

    player2_choice = "F" if i == 1 else "J"
    print("Player 2: {}".format(player2_choice))

    # player2 choice "F" all the time.
    score1, message1 = Referee.judge(i, player1.last_choice(), player2_choice)
    player1.update(score1, message1)

Round 1
Player 1: ['J']
Player 2: F
Round 2
Player 1: ['J', 'F']
Player 2: J
Round 3
Player 1: ['J', 'F', 'J']
Player 2: J
Round 4
Player 1: ['J', 'F', 'J', 'J']
Player 2: J
Round 5
retry.......(The best option cannot be determined from the information provided as the other player's choice in round 5 is not known. The assistant can make a guess but it would just be a 50/50 chance. A possible answer could be "J".)
Player 1: ['J', 'F', 'J', 'J', 'J']
Player 2: J
Round 6
retry.......(The information given does not tell me what the other player will choose in this round. Therefore, I cannot make an informed choice.)
retry.......(The information provided does not indicate a clear best strategy to choose either Option J or Option F. Therefore, the answer could be either "J" or "F".)
retry.......(The game doesn't provide a strategy for which option is better in general, it depends on the opponent's action. As an AI, I don't have a way to predict human actions. However, I can say that option F

KeyboardInterrupt: 